In [1]:
# The C2QA package is currently not published to PyPI.
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
from quspin.operators import hamiltonian  # operators
from quspin.basis import boson_basis_1d  # Hilbert space boson basis
from quspin.basis import tensor_basis, spinless_fermion_basis_1d  # Hilbert spaces
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function, division
from quspin.tools.Floquet import Floquet_t_vec
from quspin.tools.evolution import evolve  # ODE evolve tool
from quspin.operators import hamiltonian  # operators
from quspin.basis import boson_basis_1d  # Hilbert space boson basis
from quspin.basis import tensor_basis, spinless_fermion_basis_1d  # Hilbert spaces
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
import copy
import c2qa
import util
from scipy.sparse.linalg import eigsh

### Build basis and Hamiltonian

In [2]:
lab=[]
elist=[]

In [4]:
Nsites = 4
Nbosons = 4
###### parameters
L_spin = Nsites - 1
L_modes = Nsites  # system size
cutoff = Nbosons + 1  #sites+2
h = 1  # field strength
t = 1

### Build projector onto gauge conserving basis

In [5]:
P_sparse = util.Projector_to_gauge_peserving_basis(Nsites, Nbosons)

['0+0+0+4', '0+0+1-3', '0+0+2+2', '0+0+3-1', '0+0+4+0', '0+1-0-3', '0+1-1+2', '0+1-2-1', '0+1-3+0', '0+2+0+2', '0+2+1-1', '0+2+2+0', '0+3-0-1', '0+3-1+0', '0+4+0+0', '1-0-0-3', '1-0-1+2', '1-0-2-1', '1-0-3+0', '1-1+0+2', '1-1+1-1', '1-1+2+0', '1-2-0-1', '1-2-1+0', '1-3+0+0', '2+0+0+2', '2+0+1-1', '2+0+2+0', '2+1-0-1', '2+1-1+0', '2+2+0+0', '3-0-0-1', '3-0-1+0', '3-1+0+0', '4+0+0+0']


In [6]:
# building the two bases to tensor together
basis_spin = spin_basis_1d(L=L_spin)
basis_boson = boson_basis_1d(L=L_modes, sps=cutoff)
# print(basis_boson)
basis = tensor_basis(basis_spin, basis_boson)
# print(basis)
# print(basis.index("10","010"))

# Ratios of hopping and field

In [10]:
hopping_strength = -1
field_strength = 1
Hgaugefixed = util.build_H(hopping_strength, field_strength, L_modes, L_spin, P_sparse, basis)
E, psi0 = eigsh(Hgaugefixed, k=1, which='SA')
# States with weight
for i in range(len(psi0)):
    print(np.abs(psi0[i]))

[0.00462891]
[0.02265571]
[0.0435365]
[0.07984152]
[0.04331705]
[0.06056038]
[0.1238168]
[0.24744763]
[0.13216908]
[0.06350055]
[0.20738228]
[0.12606872]
[0.20540618]
[0.13216908]
[0.04331705]
[0.11345814]
[0.21321997]
[0.40280918]
[0.20540618]
[0.10833356]
[0.35107481]
[0.20738228]
[0.40280918]
[0.24744763]
[0.07984152]
[0.03130252]
[0.10833356]
[0.06350055]
[0.21321997]
[0.1238168]
[0.0435365]
[0.11345814]
[0.06056038]
[0.02265571]
[0.00462891]


In [11]:
psi0_notgaugefixed = P_sparse.T @ psi0

util.gauge_invariant_correlator(hopping_strength, field_strength, Nsites, psi0_notgaugefixed, Nbosons, basis)

[[ 1.76152093  2.23847907  2.23847907  1.76152093]
 [ 1.78583734  2.0244158   1.78583734 -1.        ]
 [ 1.49678554  1.49678554 -1.         -1.        ]
 [ 1.06337245 -1.         -1.         -1.        ]]


In [12]:
psi0_notgaugefixed = P_sparse.T @ psi0
util.pairing_correlator(hopping_strength, field_strength, Nsites, psi0_notgaugefixed, Nbosons, basis)

### Build Hamiltonian and calculate ground state

#### Hopping dominates: ground state is (1/2 |2+0> + 1/2 |0+2> + 1/sqrt(2) |1-1>)

#### Positive field dominates so |-> lowers the energy the most: ground state is |1-1>

In [13]:
E, psi0 = eigsh(Hgaugefixed, k=20, which='SA')
print(np.sort(E))
print(E[1]-E[0])
# print(psi0)
# psi0[:,6]

[-6.78878228 -5.76486707 -4.49665979 -4.2945786  -4.17908439 -3.92360838
 -2.55522867 -2.35474492 -2.27610921 -1.72895331 -1.58422544 -1.17270777
 -1.09373469 -0.55602007 -0.32487352 -0.19297192  0.29428036  0.40708276
  0.79589098  1.05406408]
1.0239152115350318


In [14]:
lab.append(str(Nsites)+" sites, "+str(Nbosons)+" bosons")
elist.append(np.sort(E))
for i in range(len(lab)):
    plt.plot(range(len(elist[i])),elist[i],".",label=lab[i])
plt.xlabel("Eigenenergy number")
plt.ylabel("Eigenenergy")
plt.legend()
plt.show()

# Energy gap

In [15]:
set=[]
lab = []

In [ ]:
res = util.energy_gap(set, lab, -1, 1, L_modes, L_spin, P_sparse)
set = res[0]
energy0 = res[1]
energy1 = res[2]

In [ ]:
lab.append(str(Nsites)+" sites, "+str(Nbosons)+" bosons")

In [ ]:
xlabelplot = "Value of the field term (hopping=1)"
plt.title("Z2LGT Energy gaps")
for i in range(len(set)):
    plt.plot(energy0[0],energy0[1],label="E0 "+lab[i])
for i in range(len(set)):
    plt.plot(energy1[0],energy1[1],label="E1 "+lab[i])
plt.xlabel(xlabelplot)
plt.ylabel("Eigenvalue")
plt.legend()
plt.show()

plt.title("Z2LGT Energy gap")
for i in range(len(set)):
    plt.plot(set[i][0],set[i][1],label=lab[i])
    # plt.plot(set[i][0],set[i][0]**(-1),".",label="$x^{-2}$")
plt.xlabel(xlabelplot)
plt.ylabel("Energy gap")
# plt.yscale("log")
# plt.xscale("log")
plt.legend()
plt.show()

#### field fixed varying hopping - scaling of energy gap

In [ ]:
set=[]
lab = []
lab.append(str(Nsites)+" sites, "+str(Nbosons)+" bosons")

###### parameters
min = -5
max = 0
numberofvalues = 100
vals=np.linspace(min,max,numberofvalues)
vals=np.logspace(-2,2,numberofvalues)

# Different system sizes and number of bosons - choose number of bosons to be equal to the system size

gj=1
deltas=np.zeros((2,len(vals)))
energy0=np.zeros((2,len(vals)))
energy1=np.zeros((2,len(vals)))
for i in range(len(vals)):
    val=vals[i]
    ##### create model
    hop=[[val,i,i,i+1] for i in range(L_modes-1)]
    # density = [[0,i,i] for i in range(L_modes)]
    field = [[-1,i] for i in range(L_spin)]
    static=[["z|+-",hop],["z|-+",hop],["x|",field]]#,["|nn",density]]
    ###### setting up operators
    # set up hamiltonian dictionary and observable (imbalance I)
    no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
    H = hamiltonian(static,[],basis=basis,**no_checks)
    H_sparse = H.tocsr()

    Hgaugefixed=P_sparse@H_sparse@P_sparse.T.conj()
    print("done ")
    E,V = eigsh(Hgaugefixed,k=2,which='SA')
    delta=np.abs(E[1]-E[0])
    if val==0:
        print("E[0] ",E[0]," E[1] ",E[1]," delta ",delta)
    deltas[0][i]=val
    deltas[1][i]=delta

    energy0[0][i]=val
    energy0[1][i]=E[0]

    energy1[0][i]=val
    energy1[1][i]=E[1]

set.append(deltas)

In [ ]:
xlabelplot = "Value of the hopping term (field=1)"
plt.title("Z2LGT Energy gaps")
for i in range(len(set)):
    plt.plot(energy0[0],energy0[1],label="E0 "+lab[i])
for i in range(len(set)):
    plt.plot(energy1[0],energy1[1],label="E1 "+lab[i])
plt.xlabel(xlabelplot)
plt.ylabel("Eigenvalue")
plt.legend()
plt.show()

plt.title("Z2LGT Energy gap")
for i in range(len(set)):
    plt.plot(np.abs(set[i][0]),np.abs(set[i][1]),label=lab[i])
    plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(2)),label="$x^{2}$")
    plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(1)),label="$x$")
plt.xlabel(xlabelplot)
plt.ylabel("Energy gap")
plt.yscale("log")
plt.xscale("log")
plt.ylim(10**(-3))

plt.legend()
plt.show()

### define initial state (ground state of X field)

In [ ]:
##### create model
field = [[-h,i] for i in range(L_spin)]
# density = [[0.01,i,i] for i in range(L_modes)]
static=[["x|",field]]#,["|nn",density]]
###### setting up operators
# set up hamiltonian dictionary and observable (imbalance I)
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
H1 = hamiltonian(static,[],basis=basis,**no_checks)

H = hamiltonian(static, [], basis=basis, **no_checks)
H_sparse = H.tocsr()
Hgaugefixed = P_sparse @ H_sparse @ P_sparse.T.conj()
_,psi = eigsh(Hgaugefixed,k=1, which='SA')
# P=P_gaugefixed.full()
# psi=np.dot(P.T.conj(),psi)

### Create |0+2> by hand and check where it is in the basis

In [ ]:
##### define initial state #####
psispin = (1 / np.sqrt(2)) * np.array([1, 1])
#Boson - find index of Fock state |20>
iboson = basis_boson.index("02")
# Ns is the size of the Hilbert space
psiboson = np.zeros(basis_boson.Ns, dtype=np.float64)
psiboson[iboson] = 1.0
psi = np.kron(psispin, psiboson)
print(psi)

In [ ]:
# Check the state which was made
statetest = [[1.0,0]]  #second index chooses which spin or mode to check (ie. 0 is the 1st mode, 1 is the second and same for spins)
static = [["|n", statetest]]  #z| checks magnetization of spins, |n checks boson number in modes
no_checks = dict(check_pcon=False, check_symm=False, check_herm=False)
H_check = hamiltonian(static, [], basis=basis, **no_checks)
print(np.dot(psi.conj().T, H_check.dot(psi)))